In [12]:
import numpy as np
import pandas as pd
from datetime import datetime

In [21]:
weather_df = pd.read_csv('weather.csv')
cab_weather_df = pd.read_csv('cab_rides.csv')

In [22]:
cab_weather_df.isnull().sum()

distance                0
cab_type                0
time_stamp              0
destination             0
source                  0
price               55095
surge_multiplier        0
id                      0
product_id              0
name                    0
dtype: int64

In [23]:
#Dropping instances where the price is null 
cab_weather_df.dropna(inplace=True)

cab_weather_df.isnull().sum()

distance            0
cab_type            0
time_stamp          0
destination         0
source              0
price               0
surge_multiplier    0
id                  0
product_id          0
name                0
dtype: int64

In [24]:
cab_weather_df.columns

Index(['distance', 'cab_type', 'time_stamp', 'destination', 'source', 'price',
       'surge_multiplier', 'id', 'product_id', 'name'],
      dtype='object')

In [25]:
cab_weather_df = cab_weather_df[['distance', 'cab_type', 'time_stamp', 'destination', 'source', 'price',
       'surge_multiplier', 'name']]

cab_weather_df.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,name
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,Shared
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,Lux
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,Lyft
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,Lux Black XL
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,Lyft XL


In [26]:
weather_df.columns = ['temp', 'source', 'clouds', 'pressure', 'rain', 'time_stamp', 'humidity', 'wind']

In [27]:
#we want to convert the timestamps into date, time, weekday, hour, and time of day (Morning, afternoon, evening, and night)
cab_weather_df["rounded_timestamp"] = cab_weather_df["time_stamp"] / 1000
cab_weather_df["rounded_timestamp"] = cab_weather_df["rounded_timestamp"].apply(np.floor)

cab_weather_df["date"] = cab_weather_df["rounded_timestamp"].apply(lambda x : datetime.fromtimestamp(x).date())
cab_weather_df["time"] = cab_weather_df["rounded_timestamp"].apply(lambda x: datetime.fromtimestamp(x).time())
cab_weather_df['weekday'] = cab_weather_df['date'].apply(lambda x: x.weekday())
cab_weather_df["weekday"] = cab_weather_df["weekday"].map({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'})
cab_weather_df['hour'] = cab_weather_df['time'].apply(lambda time: time.hour)

cab_weather_df.loc[(cab_weather_df.hour >= 6) & (cab_weather_df.hour < 12) , 'time_of_day'] = 'Morning'
cab_weather_df.loc[(cab_weather_df.hour >= 12) & (cab_weather_df.hour < 15) , 'time_of_day'] = 'Afternoon'
cab_weather_df.loc[(cab_weather_df.hour >= 15) & (cab_weather_df.hour < 18) , 'time_of_day'] = 'Evening'
cab_weather_df.loc[(cab_weather_df.hour >= 18) | (cab_weather_df.hour < 6) , 'time_of_day'] = 'Night'

cab_weather_df = cab_weather_df[['distance', 'cab_type', 'time_stamp', 'destination', 'source', 'price',
       'surge_multiplier', 'name', 'date', 'time', 'weekday', 'hour', 'time_of_day']]

In [28]:
#we want to convert the timestamps into date, time, weekday, hour, and time of day (Morning, afternoon, evening, and night)
weather_df["rounded_timestamp"] = weather_df["time_stamp"] / 1000
weather_df["rounded_timestamp"] = weather_df["rounded_timestamp"].apply(np.floor)

weather_df["date"] = weather_df["rounded_timestamp"].apply(lambda x : datetime.fromtimestamp(x).date())
weather_df["time"] = weather_df["rounded_timestamp"].apply(lambda x: datetime.fromtimestamp(x).time())
weather_df['weekday'] = weather_df['date'].apply(lambda x: x.weekday())
weather_df["weekday"] = weather_df["weekday"].map({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'})
weather_df['hour'] = weather_df['time'].apply(lambda time: time.hour)

weather_df.loc[(weather_df.hour >= 6) & (weather_df.hour < 12) , 'time_of_day'] = 'Morning'
weather_df.loc[(weather_df.hour >= 12) & (weather_df.hour < 15) , 'time_of_day'] = 'Afternoon'
weather_df.loc[(weather_df.hour >= 15) & (weather_df.hour < 18) , 'time_of_day'] = 'Evening'
weather_df.loc[(weather_df.hour >= 18) | (weather_df.hour < 6) , 'time_of_day'] = 'Night'

weather_df = weather_df[['temp', 'source', 'clouds', 'pressure', 'rain', 'time_stamp', 'humidity', 
                         'wind', 'date', 'time', 'weekday', 'hour', 'time_of_day']]

In [43]:
merged_data = pd.merge(weather_df, cab_weather_df, on=["source", 'date'], how="outer")

In [44]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 644252 entries, 0 to 644251
Data columns (total 24 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   temp              6276 non-null    float64
 1   source            644252 non-null  object 
 2   clouds            6276 non-null    float64
 3   pressure          6276 non-null    float64
 4   rain              894 non-null     float64
 5   time_stamp_x      6276 non-null    float64
 6   humidity          6276 non-null    float64
 7   wind              6276 non-null    float64
 8   date              644252 non-null  object 
 9   time_x            6276 non-null    object 
 10  weekday_x         6276 non-null    object 
 11  hour_x            6276 non-null    float64
 12  time_of_day_x     6276 non-null    object 
 13  distance          637976 non-null  float64
 14  cab_type          637976 non-null  object 
 15  time_stamp_y      637976 non-null  float64
 16  destination       63

In [42]:
merged_data.count()

temp                  6276
source              644252
clouds                6276
pressure              6276
rain                   894
time_stamp          644252
humidity              6276
wind                  6276
date_x                6276
time_x                6276
weekday_x             6276
hour_x                6276
time_of_day_x         6276
distance            637976
cab_type            637976
destination         637976
price               637976
surge_multiplier    637976
name                637976
date_y              637976
time_y              637976
weekday_y           637976
hour_y              637976
time_of_day_y       637976
dtype: int64

In [134]:
merged_data.isnull().sum()

temp                 0
source               0
clouds               0
pressure             0
rain                 0
time_stamp           0
humidity             0
wind                 0
distance            25
cab_type            25
destination         25
price               25
surge_multiplier    25
name                25
dtype: int64

In [137]:
merged_data.dropna(inplace=True)

merged_data.head()

,temp,source,clouds,pressure,rain,time_stamp,humidity,wind,distance,cab_type,destination,price,surge_multiplier,name
25,39.23,Back Bay,0.83,996.09,0.0284,1543449727,0.66,10.67,1.33,Lyft,South Station,13.5,1.0,Lux
26,39.23,Back Bay,0.83,996.09,0.0284,1543449727,0.66,10.67,3.17,Uber,North End,33.0,1.0,Black SUV
27,39.23,Back Bay,0.83,996.09,0.0284,1543449727,0.66,10.67,3.17,Uber,North End,10.0,1.0,WAV
28,39.23,Back Bay,0.83,996.09,0.0284,1543449727,0.66,10.67,3.17,Uber,North End,10.0,1.0,UberX
29,39.23,Back Bay,0.83,996.09,0.0284,1543449727,0.66,10.67,3.17,Uber,North End,22.0,1.0,Black
